Clone the repo from GitHub

In [1]:
!git clone https://github.com/Ryan-Rudes/tacotron2
%cd tacotron2
!git submodule init; git submodule update

Cloning into 'tacotron2'...
remote: Enumerating objects: 450, done.
remote: Total 450 (delta 0), reused 0 (delta 0), pack-reused 450
Receiving objects: 100% (450/450), 2.71 MiB | 27.77 MiB/s, done.
Resolving deltas: 100% (224/224), done.
/kaggle/working/tacotron2
Submodule 'waveglow' (https://github.com/NVIDIA/waveglow) registered for path 'waveglow'
Cloning into '/kaggle/working/tacotron2/waveglow'...
Submodule path 'waveglow': checked out '5bc2a53e20b3b533362f974cfa1ea0267ae1c2b1'


Create file lists from train and validation sets

In [2]:
import pandas as pd


metadata = pd.read_csv("../../input/johnoliver/metadata.csv")
metadata = metadata[metadata['include']]

total = len(metadata)
split = 0.9
train = int(total * split)

train_metadata = metadata[:train]
val_metadata = metadata[train:]

with open('filelists/audio_text_train_filelist.txt', 'w') as f:
    for _, (index, _, _, _, text, _) in train_metadata.iterrows():
        filepath = '../../input/johnoliver/wav/%05d.wav' % index
        f.write(filepath + '|' + text + '\n')

with open('filelists/audio_text_test_filelist.txt', 'w') as f:
    for _, (index, _, _, _, text, _) in val_metadata.iterrows():
        filepath = '../../input/johnoliver/wav/%05d.wav' % index
        f.write(filepath + '|' + text + '\n')

In [3]:
df = pd.read_csv("/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt", sep="|", names=["path", "text"])

Install dependencies

In [4]:
with open("/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt", "r") as f:
    lines = f.readlines()
with open("/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt", "w") as f:
    for line in lines:
        line = line.replace('DUMMY', '/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs')
        f.write(line)

In [5]:
# with open("/kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt", "r") as f:
#     lines = f.readlines()
# lines

In [6]:
with open("/kaggle/working/tacotron2/filelists/ljs_audio_text_test_filelist.txt", "r") as f:
    lines = f.readlines()
with open("/kaggle/working/tacotron2/filelists/ljs_audio_text_test_filelist.txt", "w") as f:
    for line in lines:
        line = line.replace('DUMMY', '/kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs')
        f.write(line)

In [7]:
!mv /kaggle/working/tacotron2/filelists/ljs_audio_text_train_filelist.txt /kaggle/working/tacotron2/filelists/audio_text_train_filelist.txt

In [8]:
!mv /kaggle/working/tacotron2/filelists/ljs_audio_text_test_filelist.txt /kaggle/working/tacotron2/filelists/audio_text_test_filelist.txt

In [9]:
!ls tacotron2/filelists

ls: cannot access 'tacotron2/filelists': No such file or directory


In [10]:
!pip install tensorflow==1.15
!pip install unidecode
!pip install inflect

     |████████████████████████████████| 412.3 MB 20 kB/s s eta 0:00:01     |█████████████████               | 219.2 MB 76.4 MB/s eta 0:00:03
     |████████████████████████████████| 50 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 49.3 MB/s eta 0:00:01
     |████████████████████████████████| 503 kB 48.0 MB/s eta 0:00:01
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7538 sha256=35a7d738f0465ea9761e2fd79bec7d3dacfc2a2bfea37d0665905d6ec82d865e
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4

Download pretrained model from Google Drive for transfer learning

In [11]:
!python --version

Python 3.7.9


In [12]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA

     |████████████████████████████████| 147 kB 7.1 MB/s eta 0:00:01
Downloading...
From (original): https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA
From (redirected): https://drive.google.com/uc?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA&confirm=t&uuid=634c2f93-df1d-43b6-8eb8-f1a86c32208b
To: /kaggle/working/tacotron2/tacotron2_statedict.pt
100%|████████████████████████████████████████| 113M/113M [00:01<00:00, 58.5MB/s]


Train the model for 1000 epochs with a batch size of 32

In [13]:
!apt-get update --fix-missing
!apt-get install sox -y
!soxi /kaggle/input/the-lj-speech-dataset/LJSpeech-1.1/wavs/LJ001-0001.wav

Get:1 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1581 B]
Ign:2 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [102 kB]     
Err:1 http://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
  The following signatures couldn't be verified because the public key is not available: NO_PUBKEY A4B469963BF863CC
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [102 kB]       
Get:7 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [1229 B]   
Get:8 http://packages.cloud.google.com/apt cloud-sdk-bionic InRelease [1644 B] 
Get:10 http://packages.cloud.google.com/apt cloud-sdk InRelease [1616 B]       
Get:11 ht

In [14]:
# !python train.py --output_directory=outdir --log_directory=logdir -c tacotron2_statedict.pt --warm_start
!python train.py --output_directory=outdir --log_directory=logdir -c tacotron2_statedict.pt --warm_start --hparams sampling_rate=22050,iters_per_checkpoint=100

FP16 Run: False
Dynamic Loss Scaling: True
Distributed Run: False
cuDNN Enabled: True
cuDNN Benchmark: False
Warm starting model from checkpoint 'tacotron2_statedict.pt'
Epoch: 0
Train loss 0 0.400323 Grad Norm 1.982222 4.60s/it
Validation loss 0:  2.271830  
Saving model and optimizer state at iteration 0 to outdir/checkpoint_0
Train loss 1 1.141755 Grad Norm 21.821270 3.98s/it
Train loss 2 1.076679 Grad Norm 8.233188 4.25s/it
Train loss 3 0.809708 Grad Norm 6.357800 4.08s/it
Train loss 4 0.694643 Grad Norm 3.805735 4.52s/it
Train loss 5 0.616416 Grad Norm 4.179177 4.21s/it
Train loss 6 0.652007 Grad Norm 4.546817 4.29s/it
Train loss 7 0.608955 Grad Norm 2.582602 4.20s/it
Train loss 8 0.620387 Grad Norm 2.907915 4.18s/it
Train loss 9 0.581993 Grad Norm 1.606055 4.01s/it
Train loss 10 0.541180 Grad Norm 2.521045 4.44s/it
Train loss 11 0.503215 Grad Norm 2.616086 4.15s/it
Train loss 12 0.556231 Grad Norm 2.610107 4.00s/it
Train loss 13 0.511975 Grad Norm 1.630932 4.17s/it
Train loss 14 

## Inference
### Synthesize generated audio samples from text

In [15]:
!python -m pip install spacy

In [16]:
#Trying Inflect 5.0.x series
!pip install inflect==5.0.1

  Attempting uninstall: inflect
    Found existing installation: inflect 6.0.5
    Uninstalling inflect-6.0.5:
      Successfully uninstalled inflect-6.0.5


In [17]:
from multiprocessing import Pool
import matplotlib.pylab as plt
%matplotlib inline
from tqdm.notebook import tqdm
import IPython.display as ipd
from time import time, sleep
import scipy.io.wavfile
import numpy as np
import matplotlib
import torch
import sys

sys.path.append('waveglow/')

from audio_processing import griffin_lim
from layers import TacotronSTFT, STFT
from hparams import create_hparams
from text import text_to_sequence
from denoiser import Denoiser
from train import load_model
from model import Tacotron2

In [18]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower', interpolation='none')

In [19]:
hparams = create_hparams()
hparams.sampling_rate = 22050

In [20]:
# checkpoint = int(input("Enter steps at latest checkpoint: "))
checkpoint_path = f"outdir/checkpoint_900"
# checkpoint_path = "/kaggle/working/tacotron2/tacotron2_statedict.pt"
model = load_model(hparams)
model.load_state_dict(torch.load(checkpoint_path)['state_dict'])
_ = model.cuda().eval().half()

In [21]:
!gdown https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF

Downloading...
From (original): https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF
From (redirected): https://drive.google.com/uc?id=1rpK8CzAAirq9sWZhe9nlfvxMF1dRgFbF&confirm=t&uuid=9d7b4c5c-8dfa-495d-a23e-85fc51c89278
To: /kaggle/working/tacotron2/waveglow_256channels_universal_v5.pt
100%|█████████████████████████████████████████| 676M/676M [00:06<00:00, 107MB/s]


In [22]:
waveglow_path = 'waveglow_256channels_universal_v5.pt'
waveglow = torch.load(waveglow_path)['model']
waveglow.cuda().eval().half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/opt/conda/lib/python3.7/site-packages/torch/serialization.py:658: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv1d' has changed. you can retrieve the original source code by accessing the object's source attribute or

In [23]:
def synthesize(word, n=5, cleaners=['english_cleaners'], sigma=0.666, strength=0.01):
    sequence = np.array(text_to_sequence(word, cleaners))[None, :]
    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
    with torch.no_grad():
        mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
    audio = waveglow.infer(mel_outputs_postnet, sigma=sigma)
    audio_denoised = denoiser(audio, strength=strength)[:, 0]
    audio = audio[0].data.cpu().numpy().tolist()
    mel_outputs = mel_outputs.float().data.cpu().numpy()[0]
    mel_outputs_postnet = mel_outputs_postnet.float().data.cpu().numpy()[0]
    alignments = alignments.float().data.cpu().numpy()[0]
    return audio, mel_outputs, mel_outputs_postnet, alignments

In [26]:
tests = [
#          "Scientists at the CERN laboratory say they have discovered a new particle.",
#          "The state of Florida reports a surge in coronavirus deaths as restrictions are upended.",
#          "How much wood would a woodchuck chuck if a woodchuck could chuck wood?",
         "A woodchuck would chuck all the wood it could chuck if a woodchuck could chuck wood.",
#          "Peter Piper picked a peck of pickled peppers. How many pickled peppers did Peter Piper pick?",
#          "Sally sells seashells by the seashore. The shells she sells are seashells I'm sure.",
#          "The blue lagoon is a nineteen eighty American romance adventure film.",
#          "George Washington was the first President of the United States.",
#          "Basilar membrane and otolaryngology are not auto-correlations.",
#          "Biden holds first foreign meeting with Canada's Justin Trudeau."
]

In [27]:
for text in tests:
    audio, mel_outputs, mel_outputs_postnet, alignments = synthesize(text, n=15)
    ipd.display_html(ipd.HTML(f"""
    <h3>{text}</h3>
    <br/>
    """))
    ipd.display(ipd.Audio(data = audio, rate = 22050, autoplay = False))

A woodchuck would chuck all the wood it could chuck if a woodchuck could chuck wood.